# Box Office Mojo - Web scraping

In [122]:
# if needed: pip install requests
import requests
import urllib2
#from BeautifulSoup import BeautifulSoup
from bs4 import BeautifulSoup
import re
import dateutil.parser
from string import ascii_uppercase
from pprint import pprint
import pandas as pd
import pickle
import time

In [123]:
def get_movie_value(soup, field_name):
    '''Grab a value from boxofficemojo HTML
    
    Takes a string attribute of a movie on the page and
    returns the string in the next sibling object
    (the value for that attribute)
    or None if nothing is found.
    '''
    obj = soup.find(text=re.compile(field_name))
    if not obj: 
        return None
    # this works for most of the values
    next_sibling = obj.findNextSibling()
    if next_sibling:
        return next_sibling.text.encode('ascii','ignore') 
    else:
        return None

In [124]:
def get_movie_title(soup):
    obj = soup.find('title')
    if not obj: 
        return None
    # this works for most of the values
    try:
        name = "(".join(obj.text.split('(')[:-1]).strip()
        if name == "":
            name = "".join(obj.text.split('-')[:-1]).strip()
        return name.encode('ascii','ignore')
    except:
        return None

In [125]:
def get_theaters(soup):
    nonBreakSpace = u'\xa0'
    obj = soup.find(text=re.compile('Widest'+nonBreakSpace+'Release:'))
    if not obj:
        return None
    next_obj = obj.findNext('td')
    if next_obj.contents[0]:
        return next_obj.contents[0].strip().split()[0].encode('ascii','ignore')
    else:
        return None

In [126]:
def get_the_first_player(soup, field_name_list):
    #    actors_list = []
#    for link in soup.find_all('a'):
#        if link.contents[0] in set(field_name_list):
#            my_td = link.findNext('td')
#            my_list = my_td.find_all('a')
#            for item in my_list:
#                actors_list.append(item.text.encode('ascii','ignore'))
#            return actors_list
    return None

In [127]:
def get_all_players(soup, field_name_list):
    for item in set(field_name_list):
        my_text = soup.find(text=item)
        if my_text:
            my_td = my_text.findNext('td').getText(separator=u',').encode('ascii','ignore')
            return my_td
    return None

In [128]:
def to_date(datestring):
    try:
        date = dateutil.parser.parse(datestring)
        return date
    except:
        return datestring

def money_to_int(moneystring):
    try:
        moneystring = moneystring.replace('$', '').replace(',', '')
        return int(moneystring)
    except:
        return moneystring

def runtime_to_minutes(runtimestring):
    try:
        runtime = runtimestring.split()
        try:
            minutes = int(runtime[0])*60 + int(runtime[2])
            return minutes
        except:
            return None
    except:
        return runtimestring

In [129]:
def process_movie(url):
    headers = ['movie_title', 'domestic_total_gross', 'release_date', 'runtime_(mins)', 'rating',
               'genre', 'distributor', 'director', 'producer', 'production_budget', 'widest_release_theaters',
              'actors', 'writers', 'cinematographers', 'composers']
    response = requests.get(url)
    if response.status_code != 200:
        return None
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    movie_title = get_movie_title(soup)
    raw_release_date = get_movie_value(soup,'Release Date')
    release_date = to_date(raw_release_date)
    raw_domestic_total_gross = get_movie_value(soup,'Domestic Total')
    domestic_total_gross = money_to_int(raw_domestic_total_gross)
    raw_runtime = get_movie_value(soup,'Runtime')
    runtime = runtime_to_minutes(raw_runtime)
    rating = get_movie_value(soup,'MPAA Rating')
    genre = get_movie_value(soup,'Genre: ')
    distributor = get_movie_value(soup,'Distributor: ')
    production_budget = get_movie_value(soup, 'Production Budget: ')
    theaters = get_movie_value(soup, 'Wildest Release: ')
    director = get_all_players(soup,['Director:','Director'])
    producer = get_all_players(soup,['Producer:','Producers:','Producer','Producers'])
    actors = get_all_players(soup,['Actor:','Actors:','Actor','Actors'])
    writers = get_all_players(soup,['Writer:','Writers:','Screenwriter:','Screenwriters:',
                                   'Writer','Writers','Screenwriter','Screenwriters'])
    cinematographers = get_all_players(soup, ['Cinematographer:','Cinematographer',
                                              'Cinematographers:''Cinematographers'])
    composers = get_all_players(soup, ['Composer:','Composers:','Composer','Composers'])
    widest_release_theaters = get_theaters(soup)
    df_movie = pd.DataFrame([[movie_title, domestic_total_gross, release_date, runtime, rating, 
                              genre, distributor, director, producer, production_budget, widest_release_theaters,
                             actors, writers, cinematographers, composers]], 
                            columns=headers)
    return df_movie


In [130]:
def get_links_to_movies(main_page, movies_page):
    response = requests.get(movies_page)
    page = response.text
    soup = BeautifulSoup(page,"lxml")
    obj = soup.find(id="body")
    list_links = []
    for link in obj.findAll('a'):
        if link['href'].startswith("/movies/?id"):
            list_links.append(main_page + link['href'])
    return list_links

In [131]:
df_sample = process_movie('http://www.boxofficemojo.com/movies/?id=behavingbadly.htm')
#df_sample = df_sample.append(process_movie('http://boxofficemojo.com/movies/?id=biglebowski.htm'),ignore_index=True)
#df_sample = df_sample.append(process_movie('http://www.boxofficemojo.com/movies/?id=jazbaa.htm'),ignore_index=True)
print df_sample
#df_sample.to_csv("./data/test_sample_by_movies.csv", index = False)

None


In [132]:
main_page = 'http://www.boxofficemojo.com'

# Processing 'NUM' movies page
movies_page = main_page + '/movies/alphabetical.htm?letter=NUM&p=.htm'
print movies_page
movies_links = get_links_to_movies(main_page, movies_page)
df = process_movie(movies_links.pop(0))
for link in movies_links:
    df_movie = process_movie(link)
    if df_movie is not None:
        df = df.append(df_movie, ignore_index=True)
df.to_pickle('my_df.pickle')


http://www.boxofficemojo.com/movies/alphabetical.htm?letter=NUM&p=.htm


In [243]:
# Processing pages
for c in ascii_uppercase[25:26]:
    for num in range(1,15):
        movies_page = main_page + '/movies/alphabetical.htm?letter=' + c + '&page=' + str(num) + '&p=.htm'
        print movies_page
        movies_links = get_links_to_movies(main_page, movies_page)
        for link in movies_links:
            df = pd.read_pickle('my_df.pickle')
            df_movie = process_movie(link)
            if df_movie is not None:
                df = df.append(df_movie, ignore_index=True)
            df.to_pickle('my_df.pickle')
    #time.sleep(20)
    

http://www.boxofficemojo.com/movies/alphabetical.htm?letter=Z&page=1&p=.htm
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=Z&page=2&p=.htm
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=Z&page=3&p=.htm
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=Z&page=4&p=.htm
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=Z&page=5&p=.htm
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=Z&page=6&p=.htm
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=Z&page=7&p=.htm
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=Z&page=8&p=.htm
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=Z&page=9&p=.htm
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=Z&page=10&p=.htm
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=Z&page=11&p=.htm
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=Z&page=12&p=.htm
http://www.boxofficemojo.com/movies/alphabetical.htm?letter=Z&page=13&p=.htm
http://w

In [244]:
df.to_csv("/home/lucia/lucia/metis/02-luther/data/movies_mojo_uptoZ.csv", index = False)

In [245]:
print ascii_uppercase[25:26]
print len(df)
print df.head()
print df.tail()

Z
16811
            movie_title domestic_total_gross         release_date  \
0               #Horror                 None  2015-11-20 00:00:00   
1                 $9.99                52384  2008-12-12 00:00:00   
2       $upercapitalist                15919  2012-08-10 00:00:00   
3  (500) Days of Summer             32391374  2009-07-17 00:00:00   
4            (Untitled)               230600  2009-10-23 00:00:00   

  runtime_(mins)   rating      genre       distributor   director  \
0             90  Unknown     Horror               IFC       None   
1             78        R  Animation  Regent Releasing       None   
2             96  Unrated   Thriller       Truly Indie       None   
3             95    PG-13    Romance   Fox Searchlight  Marc Webb   
4             96        R     Comedy    Samuel Goldwyn       None   

                                    producer production_budget  \
0                                       None               N/A   
1                             